# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver - Chrome = ChromeDriver

In [1]:
# entrar na internet 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()
# Google is being controlled by automated test software.

# pegar cotação do dólar 
# entrar no google - pesquisar cotação dolar - pegar a cotação do dólar.

navegador.get("https://www.google.com/search?q=cota%C3%A7%C3%A3o+dolar")

cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)

#  pesquisar cotação euro - pegar a cotação do euro.

navegador.get("https://www.google.com/")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

#  pesquisar cotacao ouro 

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")

print(cotacao_ouro)

navegador.quit() #fecha o navegador no final da operacao 

5.2711
6.177755555
300.88


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
# importar e atualizar a base de dados (tabela que está nessa pasta)

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
#   atualizar a cotação

#.loc permite selecionar celulas da tabela pra atualizar, utlilzando linha e coluna

tabela.loc[tabela["Moeda"] == "Dólar","Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro","Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro","Cotação"] = float(cotacao_ouro)

# o python permite editar varias linhas de uma tabela só com uma linha de código

#   atualizar o preço de compra - preço original * cotação = preço final

tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

#   atualizar o preço de venda - preço de compra * margem

tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

#o pandas ja entende que é pra mudar a tabela toda

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.271100,5271.047289,1.40,7379.466205
1,Carro Renault,4500.00,Euro,6.177756,27799.899998,2.00,55599.799995
2,Notebook Dell,899.99,Dólar,5.271100,4743.937289,1.70,8064.693391
3,IPhone,799.00,Dólar,5.271100,4211.608900,1.70,7159.735130
4,Carro Fiat,3000.00,Euro,6.177756,18533.266665,1.90,35213.206663
5,Celular Xiaomi,480.48,Dólar,5.271100,2532.658128,2.00,5065.316256
6,Joia 20g,20.00,Ouro,300.880000,6017.600000,1.15,6920.240000


### Agora vamos exportar a nova base de preços atualizada

In [4]:
# exportar

tabela.to_excel("Produtos Novo.xlsx", index=False)
#tabela.to_excel("Produtos.xlsx") # se passar o nome igual ele vai substituir o arquivo
#(index=false pra aquele indice sumir)